![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Vector Search with Redispy
## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/00_redispy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Prepare data

In this examples we will load a list of movie objects with the following attributes: `title`, `rating`, `description`, and `genre`. 

For the vector part of our vector search we will embed the description so that user's can search for movies that best match what they're looking for.

**If you are running this notebook locally**, FYI you may not need to perform this step at all.

In [5]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/vector-search/resources .
!rm -rf temp_repo

Cloning into 'temp_repo'...
remote: Enumerating objects: 204, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 204 (delta 37), reused 24 (delta 24), pack-reused 152
Receiving objects: 100% (204/204), 9.47 MiB | 10.76 MiB/s, done.
Resolving deltas: 100% (64/64), done.
mv: temp_repo/python-recipes/vector-search/resources: No such file or directory


## Packages

In [2]:
# NBVAL_SKIP
%pip install -q redis numpy sentence-transformers


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [21]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

### Create redis client

In [22]:
from redis import Redis
client = Redis.from_url(REDIS_URL)

In [23]:
import json

with open("resources/movies.json", 'r') as file:
    movies = json.load(file)

In [24]:
import numpy as np
from sentence_transformers import SentenceTransformer

# load model for embedding our movie descriptions
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def embed_text(model, text):
    return np.array(model.encode(text)).astype(np.float32).tobytes()

/Users/robert.shelton/.pyenv/versions/3.11.9/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
# Note: convert embedding array to bytes for storage in Redis Hash data type
movie_data = [
    {
        **movie,
        "vector": embed_text(model, movie["description"])
    } for movie in movies
]

In [26]:
movie_data[0]

{'title': 'Explosive Pursuit',
 'genre': 'action',
 'rating': 7,
 'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.',
 'vector': b'\x9bf|=\x0e`\n;"\x92\xb7;<\xcb~\xbd\xfad\xce\xbb\xc3\x16J=V\xa7?=\xedv\x95<d\xfa\x06\xbe\x14Y\xcf=(\x07p=?\xdb\r\xbd\x95\xf2H\xbdje\xc6<E\xdfa=z8\x16\xbc\x00\xd4\x13<>\xaa\x1c=\xfd\xee\x89<\xbd\xb0-<\x82\xb2\x9f\xbc[\x0b\xc3\xbd\x98NR=xl\xf7\xbcN>\x17\xbe#\x12\x05\xb99u\xbf<\xb0\xe0b\xba\xd3\xa6\xa8\xbdx\xdc\xec\xbcRc%=\xe4\xe7r\xbb\x1eOG=?(\x85=o@\xa2\xbc2Z\xd0\xbdC%K\xbd\xb9\xed\x94\xbcR\xddH=\x92&F<\xc6*\xec<\x90\xd8\x8d\xbd\xcbZ\x98<\t\xa3\xa3=>g3\xbd&\xcd\xbd\xbd\x95$\xf7;\xfd\xf4z=\xfc\xb4\x8c=\x85\x0e\xc6\xbdnI\x90\xbdJ\x16\xbd;s\xe7\x0c\xbd 3\xc9\xbc\x85\xf8\xbb\xbc\xbf&u\xbb5\x8f\xca<\x05\x80J=\x0f\xaf*=\x8bOU\xbd\xc8\xf0\x95\xbc\x1d\x02\x19=)\xf4K<\xcb\xc2\t=F\x83\xac=\x9f\xd7\xb8\xbd\xf2\xb5\x9c\xbdB\x85\x18=\x96d&=-3\xf8<\xfa\xf7\x88<\x16v\xf2\xbb-=[\xbd\xf7\xac\xee\xbb5:A\xbd\xd9d\x

## Define Redis index schema

In [27]:
from redis.commands.search.field import VectorField, TagField, NumericField, TextField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

index_name = "movies"

schema = (
    VectorField(
        "vector",
        "HNSW",
        {
            "TYPE": "FLOAT32",
            "DIM": 384,
            "DISTANCE_METRIC": "COSINE"
        }
        ),
        NumericField("rating"),
        TagField("genre"),
        TextField("title"),
        TextField("description")
)

try:
    client.ft(index_name).info()
    print("Index exists!")
except:
    # index Definition
    definition = IndexDefinition(index_type=IndexType.HASH)

    # create Index
    client.ft(index_name).create_index(fields=schema, definition=definition)

## Populate index

In [28]:
def load_docs(client: Redis, data: list[dict]):
    for i, d in enumerate(data):
        client.hset(
            i,
            mapping = d
        )

def print_results(res):
    docs = [(doc.title, doc.genre, doc.rating) for doc in res.docs]
    print(f"Top {len(docs)} movies: ", docs)

In [29]:
load_docs(client, movie_data)

In [30]:
res = client.ft(index_name).search("*")
res

Result{20 total, docs: [Document {'id': '0', 'payload': None, 'rating': '7', 'title': 'Explosive Pursuit', 'genre': 'action', 'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.', 'vector': 'f|=\x0e`\n;";<~dλ\x16J=V?=v<d\x06\x14Y=(\x07p=?\rHje<Ea=z8\x16\x00\x13<>\x1c=<-<[\x0býNR=xlN>\x17#\x12\x059u<bӦxRc%=r\x1eOG=?(=o@2ZнC%K피RH=&F<*<؍Z<\t=>g3&ͽ$;z==\x0eƽnIJ\x16;s\x0c 3ɼ&u5<\x05J=\x0f*=OU\x1d\x02\x19=)K<\t=F=\u05f8\U000b573dB\x18=d&=-3<<\x16v-=[5:Ad\x19rd!x;;O<1,=^\x00-\x1a\x06a\x1a=̿=\x1f\x150=\U0005df29GK= =I"eF:\x1c=<o\x16]p<<9s<&4&<o\x1c\x18<\x18-<\x11N=/=/<\x0eL\x13<n\x0fκTܼ8\x05=\x11<hR1-\x1d\\91=WV=8Gּ"S=^<\x13<y/\x06p<-Н<J\x02=\x18+e\x7fQK>\x10vt\x00ɹhk=\r<3f]=\x15 4/{fQv=\x11\x0cq<,\x1c\x01\t<nt<\x19=v4m^\nV\x0eFU;G|W\x7f*y={\x10;\x15!\x0eӴ=\x15&\x15M=m$:</<\x1f\x08\x00i5\x01<\x06=4Z|!:\x1cŹ>v\x1bF?\x14ŏ(O\x17<\x12=\x05O\x7fp!Ğe9s;[a;B#:=\x15*<~\x1b=\x01V\r=*\x06=\x18u\x02<;9/=\x85=u\x0b"=i<4c=2\x08\x12;=,VW;\x15b<<r;

## Index loaded now we can perform vector search

### basic vector search

In [31]:
from redis.commands.search.query import Query

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

# Note: dialect 2 and above required for vector search
query = Query("(*)=>[KNN 3 @vector $vec_param AS dist]").sort_by("dist").dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)


Top 3 movies:  [('Fast & Furious 9', 'action', '6'), ('Despicable Me', 'comedy', '7'), ('The Incredibles', 'comedy', '8')]


### Hybrid filter vector search

Redis allows you to combine filter searches on fields within the index object allowing us to create more specific searches.

In [32]:
# Search for top 3 movies specifically in the action genre

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

# Note: genre is a tag field in our schema so the syntax is @<field_name>:{ <tag> | <tag> | ...}
query = Query("(@genre:{action})=>[KNN 3 @vector $vec_param AS dist]").sort_by('dist').dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 3 movies:  [('Fast & Furious 9', 'action', '6'), ('Mad Max: Fury Road', 'action', '8'), ('Explosive Pursuit', 'action', '7')]


In [33]:
# Search for top 3 movies specifically in the action genre with ratings at or above a 7

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

query = Query("(@genre:{action} & (@rating:[7 inf]))=>[KNN 3 @vector $vec_param AS dist]").sort_by('dist').dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 3 movies:  [('Mad Max: Fury Road', 'action', '8'), ('Explosive Pursuit', 'action', '7'), ('The Avengers', 'action', '8')]


In [34]:
# Search with full text search for movies that directly mention "criminal mastermind" in the description

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

query = Query("(@description:(criminal mastermind))=>[KNN 3 @vector $vec_param AS dist]").sort_by('dist').dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 2 movies:  [('Despicable Me', 'comedy', '7'), ('The Dark Knight', 'action', '9')]


In [35]:
# Vector search with wild card match

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

query = Query("(@description:(crim*))=>[KNN 3 @vector $vec_param AS dist]").sort_by('dist').dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 3 movies:  [('Despicable Me', 'comedy', '7'), ('The Incredibles', 'comedy', '8'), ('Explosive Pursuit', 'action', '7')]


In [42]:
# Vector search with fuzzy match

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

# Note: fuzzy match is based on Levenshtein distance. Therefore, "hero" might return result for "her" as an example.
# See docs for more info https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/query_syntax/
query = Query("(@description:%hero%)=>[KNN 3 @vector $vec_param AS dist]").sort_by('dist').dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 3 movies:  [('The Avengers', 'action', '8'), ('Black Widow', 'action', '7'), ('The Princess Diaries', 'comedy', '6')]


## Range queries

Range queries allow you to set a pre defined "threshold" for which we want to return documents. This is helpful when you only want documents with a certain distance from the search query.

In [37]:
user_query = "Family friendly fantasy movies"

embedded_user_query = embed_text(model, user_query)

query = (
    Query("@vector:[VECTOR_RANGE $radius $vector]=>{$YIELD_DISTANCE_AS: vector_distance}")
     .sort_by("vector_distance")
     .return_fields("title", "rating", "genre", "vector_distance")
     .dialect(2)
)

# Find all vectors within 0.8 of the query vector
query_params = {
    "radius": 0.8,
    "vector": embedded_user_query
}

res = client.ft(index_name).search(query, query_params)
print_results(res)


Top 6 movies:  [('The Incredibles', 'comedy', '8'), ('Black Widow', 'action', '7'), ('Despicable Me', 'comedy', '7'), ('Shrek', 'comedy', '8'), ('Monsters, Inc.', 'comedy', '8'), ('Aladdin', 'comedy', '8')]


Like the queries above, we can also chain additional filters and conditional operators with range queries. The following adds an `or` condition that returns vector search within the defined range or with a rating at or above 9.

In [38]:
user_query = "Family friendly fantasy movies"

embedded_user_query = embed_text(model, user_query)

query = (
    Query("@rating:[9 +inf] | @vector:[VECTOR_RANGE $radius $vector]=>{$YIELD_DISTANCE_AS: vector_distance}")
     .sort_by("vector_distance")
     .return_fields("title", "rating", "genre", "vector_distance")
     .dialect(2)
)

# Find all vectors within 0.8 of the query vector
query_params = {
    "radius": 0.7,
    "vector": embedded_user_query
}

res = client.ft(index_name).search(query, query_params)
print_results(res)

Top 3 movies:  [('The Incredibles', 'comedy', '8'), ('The Dark Knight', 'action', '9'), ('Inception', 'action', '9')]


In [39]:
# clean up!
client.flushall()